<a href="https://colab.research.google.com/github/windinherhair/fs_co_tr/blob/master/FS_%E5%85%A8%E9%83%A8_HiR_CUDA9_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deepfakes/faceswap (Original_HighRes 全部やる用)

## **（重要）colabの12時間&90分ルール対策**

1.新しいインスタンスを起動してから12時間経つと全てリセットされる。（GoogleDriveに学習したmodelや抽出した顔を退避させます）

2.90分アイドル状態が続くとノートの接続が外れるため、** Chromeのアドオンを使って60分置きにページをリロードする。（[Auto Refresh](https://chrome.google.com/webstore/detail/auto-refresh/ifooldnmmcmlbdennkpdnlnbgbmfalko)）**



### 事前準備

**1.[cuDNN](https://developer.nvidia.com/cudnn) 7.41 for CUDA 9.0 (cuDNN Library for Linux)をダウンロードし、GoogleDriveにアップしておく。要登録。バージョン間違える人多いので注意。**※windowsPCの場合拡張子がおかしくなるかも。拡張子をtgzに直してください。

2.体側の人の2分くらいの動画（videoA.mp4）、洗練されたBの顔セット（alignB）フォルダを用意する。（なければなるべく同じ色調、角度の顔側の人の動画10分くらい（videoB.mp4）を用意する。）※()内の名前にリネームしてください。


3.Google DriveにvideoA.mp4、alignBフォルダ、(なければvideoB.mp4）をアップロード



4.(推奨)学習済みmodelがある場合は、modelフォルダをGoogleDriveにアップロード

### その他
コメントアウト（#）をコードの前につけるとコードを呼び飛ばします。先に自分の条件に合わせてつけたり外したりしてから実行してください。**デフォルトだとBの顔セットがある人向けになってます。**
また、二回目からはDriveにalign(img)ができてるので、ffmpegとextractに(#)つけて、trainからやって下さい。

faceswap環境が整うまで30分かかります。

### **（設定）ランタイムのタイプをGPUに変更する** 

---



準備が整ったら、

**ランタイム→すべてのセルを実行**


---





In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

**Google Driveをマウント**（※ここだけ要入力）


###  URLをクリックし、そのまま進めたら出てくるコードをコピー、枠にペーストしEnter




In [0]:
from google.colab import drive
drive.mount('/content/drive')

 元からインストールされているCUDA9.2を削除

In [0]:
!rm -rf /usr/local/cuda*
!apt-get purge nvidia*
!apt-get autoremove
!apt-get autoclean

CUDA9.0をインストール

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb -O cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get -o Dpkg::Options::="--force-overwrite" install cuda-9-0 cuda-drivers

Driveからcudnn7.4をコピペして展開

In [0]:
!cp drive/'My Drive'/cudnn-9.0-linux-x64-v7.4.1.5.tgz ./

In [0]:
!tar -xf cudnn-9.0-linux-x64-v7.4.1.5.tgz

In [0]:
!cp -R cuda/include/* /usr/local/cuda-9.0/include

In [0]:
!cp -R cuda/lib64/* /usr/local/cuda-9.0/lib64

CUDAのパスを通す。

In [0]:
!echo 'export PATH=/usr/local/cuda-9.0/bin${PATH:+:${PATH}}' >> ~/.bashrc
!echo 'export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}' >> ~/.bashrc

In [0]:
!source ~/.bashrc
!ldconfig

dlibのインストール

In [0]:
!apt update
!apt install -y cmake
#!git clone https://github.com/davisking/dlib.git
#!python setup.py install --yes USE_AVX_INSTRUCTIONS --yes DLIB_USE_CUDA
!apt install -y --no-install-recommends cmake && pip install dlib

TFをダウングレード

In [0]:
!pip install tensorflow-gpu==1.11.0

ffmpegをインストール

In [0]:
!apt-get install -y ffmpeg

Githubのdeepfakes/faceswapをクローン

In [0]:
!git clone https://github.com/deepfakes/faceswap.git

faceswapディレクトリに移動

In [0]:
cd faceswap

残りの必要なものをインストール

In [0]:
!pip install -r requirements.txt*



---




>  ここでやっとFaceswap環境が整う。




> ```
!python faceswap.py -h
```


> でコマンドを見れるよ。


---









In [0]:
#!python faceswap.py -h

In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

## Aの顔セット準備

素材をドライブからコピー

In [0]:
!cp /content/drive/'My Drive'/videoA.mp4 /content/faceswap/



---


ffmpegでvideoAを画像にする。(1秒あたり25フレーム)

In [0]:
!mkdir /content/drive/'My Drive'/imgA  # 出力先ディレクトリを作成しておく

In [0]:

!ffmpeg -i /content/faceswap/videoA.mp4 -vf fps=25 /content/drive/'My Drive'/imgA/imgA%06d.png



---


**Extract_A**(mtcnn)


In [0]:
!python faceswap.py extract -i /content/drive/'My Drive'/imgA/ -o /content/drive/'My Drive'/alignA/ -D mtcnn -A fan -mp

Alignments tool で複数検知したものを別のフォルダに

In [0]:
!python tools.py alignments -j multi-faces -a /content/drive/'My Drive'/imgA/alignments.json -o move -fc /content/drive/'My Drive'/alignA 

## →別のタブでGoogleDriveのalignA/multiple_faces_～フォルダを開いて手動で顔以外を消す

In [0]:
input() #実行一時停止用、顔の削除が終わったら枠の中でEnterを押して下さい！

In [0]:
移した画像をalignAに戻す

In [0]:
mv /content/drive/'My Drive'/alignA/multiple_faces_in_frame_*?/0/* /content/drive/'My Drive'/alignA/multiple_faces_in_frame_*?/1/* /content/drive/'My Drive'/alignA 

## Bの顔セット準備 

コメントアウト(#)外して使ってください

In [0]:
#cp /content/drive/'My Drive'/videoB.mp4 /content/faceswap/


ffmpegでvideoBを画像にする。(1秒あたり5フレーム)



In [0]:
#!mkdir /content/faceswap/imgB  # 出力先ディレクトリを作成しておく

In [0]:

#!ffmpeg -i /content/faceswap/videoB.mp4 -vf fps=5 /content/faceswap/imgB/imgB%06d.png


**Extract_B**(mtcnn)



In [0]:
#!python faceswap.py extract -i /content/faceswap/imgB/ -o /content/drive/'My Drive'/alignB/ -D mtcnn -A fan -mp

誤検知を消しやすいようにalignBを並びかえる。

In [0]:

#!python tools.py sort -i /content/drive/'My Drive'/alignB/ -fp rename -s hist



## →別のタブでGoogleDriveのalignBフォルダを開いて手動で顔以外を消す



---



※便利な**tools**もあるよ

In [0]:
#!python tools.py sort -h

In [0]:
#!python tools.py alignments -h



---


## (選択)**Train**(バッチ32で100回ごとに保存)※設定はお好みで

プレビューが見れないので6時間くらいを目安にやってみてください。※事前学習済みmodelがある前提です。

コメントアウト(#)外して使ってください

In [0]:
#!python faceswap.py train -A /content/drive/'My Drive'/alignB/ -B /content/drive/'My Drive'/alignB/ -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32
#BとBの事前学習

In [0]:
#!python faceswap.py train -A /content/drive/'My Drive'/alignA/ -B /content/drive/'My Drive'/alignB/ -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32
#AとBの通常学習

In [0]:
#オプションはお好みで
#python faceswap.py train -h



---


**Convert**



In [0]:
#!python faceswap.py convert -i /content/drive/'My Drive'/imgA/ -o /mergeA -al /content/drive/'My Drive'/imgA/alignments.json -m /content/drive/'My Drive'/model/ -a /content/drive/'My Drive'/alignA/ -t OriginalHighRes -c Masked -e 20 -b 30 -M facehull -mh

In [0]:
#オプションはお好みで
#!python faceswap.py convert -h



---

ffmpegで画像から動画（merged_videoA.mp4）に変換

In [0]:
#!ffmpeg -framerate 25 -i /mergeA/imgA%06d.png -vcodec libx264 -pix_fmt yuv420p -r 30 /content/drive/'My Drive'/merged_videoA.mp4

# 終了～